In [1]:
import os
import glob2
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt

In [11]:
base_dir = '/home/xing/Share/Projects/Sanmed/cell_seg/20201127'

In [2]:
train_dir = '/home/xing/Share/Projects/Sanmed/cell_seg/20201127/train/DAPI-Low'
valid_dir = '/home/xing/Share/Projects/Sanmed/cell_seg/20201127/valid/DAPI-Low'

In [3]:
train_file_list = glob2.glob(os.path.join(train_dir,'*'+'.png'))
valid_file_list = glob2.glob(os.path.join(valid_dir,'*'+'.png'))

In [4]:
len(valid_file_list)

10

In [5]:
key_fn = lambda x: os.path.split(x)[-1].split('.')[0]

In [8]:
ID = []
mask_path = []
image_path = []

for mask_file in train_file_list:
    
    sid = key_fn(mask_file)
    ID.append(sid)
    mask_path.append(mask_file)
    img_pth = os.path.join(train_dir,sid+'.jpg')
    image_path.append(img_pth)

In [9]:
base_df = pd.DataFrame({'sid':ID,
                       'image_path':image_path,
                       'mask_path':mask_path})

In [10]:
base_df.head()

,sid,image_path,mask_path
0,897068_899051_10000,/home/xing/Share/Projects/Sanmed/cell_seg/2020...,/home/xing/Share/Projects/Sanmed/cell_seg/2020...
1,1245106_1249759_10000,/home/xing/Share/Projects/Sanmed/cell_seg/2020...,/home/xing/Share/Projects/Sanmed/cell_seg/2020...
2,809842_806027_10000,/home/xing/Share/Projects/Sanmed/cell_seg/2020...,/home/xing/Share/Projects/Sanmed/cell_seg/2020...
3,1408274_1406233_10000,/home/xing/Share/Projects/Sanmed/cell_seg/2020...,/home/xing/Share/Projects/Sanmed/cell_seg/2020...
4,753802_751973_10000,/home/xing/Share/Projects/Sanmed/cell_seg/2020...,/home/xing/Share/Projects/Sanmed/cell_seg/2020...


In [12]:
base_df.to_csv(os.path.join(base_dir,'train_df.csv'))

In [13]:
def mask_generation(mask,folder):
    from tqdm import tqdm
    
    if isinstance(mask,str):
        mask = cv2.imread(mask)
        
    mask = np.array(mask,np.uint8)
    mask_gray = mask[:,:,0].copy()
    uniq = np.unique(mask_gray)
    instances = np.delete(uniq,[0])
    for inst in instances:
#         print(inst)
        mask_new = np.zeros_like(mask_gray)
#         print(mask_new.shape)
        mask_new[mask_gray==inst] = 100
        
        contours,hierarchy = cv2.findContours(mask_new,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
        for idx,contour in enumerate(contours):
            mask_sub = np.zeros_like(mask_gray)
            area = cv2.contourArea(contour)
#             print(area)
            if area > 5:
                cv2.drawContours(mask_sub,contours,idx,255,-1)
                mask_sub = np.where(mask_sub>0,100,0)
                mask_nm = os.path.join(folder,'mask_'+str(inst)+'_'+str(idx)+'.png')
#                 print(mask_nm)
                cv2.imwrite(mask_nm,mask_sub)

In [14]:
def generate_folder(output,idx):
    
    img_folder = os.path.join(output,idx,'images') 
    msk_folder = os.path.join(output,idx,'masks') 
    
    if not os.path.exists(img_folder):
        os.makedirs(img_folder)
    if not os.path.exists(msk_folder):
        os.makedirs(msk_folder)
        
    return img_folder,msk_folder

In [32]:
def image_pad(img,new_shape):
    w,h,_ = img.shape
    (w_new,h_new) = new_shape
    
    pad = (w_new-w,h_new-h)
    
#     print(pad[0]//2,w_new-(pad[0]//2))
    imgNew = np.zeros((w_new,h_new,3))
    imgNew[pad[0]//2:w_new-(pad[0]//2),pad[1]//2:h_new-(pad[1]//2),:] = img
    
    return imgNew

In [31]:
# generate the data according to kaggle format with pathes

import shutil
from skimage.util.shape import view_as_windows
from tqdm import tqdm

# input_dir = r''
input_dir = r'/home/xing/Share/Projects/Sanmed/cell_seg/20201127/train/DAPI-Low'
output_dir = r'/home/xing/Share/Projects/Sanmed/cell_seg/20201127/Dapi_output/train'
w = 2560
h = 2560
pw = 512
ph = 512

file_list = glob2.glob(os.path.join(input_dir,'*'+'.png'))

for mask_pth in tqdm(file_list):
    
    sid = key_fn(mask_pth)
    
#     print(sid,mask_pth)
    
    # step 1: image readin, resize, and patch generation
    
    img_pth = os.path.join(input_dir,sid+'.jpg')
#     print(img_pth)
    
    image = cv2.imread(img_pth)
    mask = cv2.imread(mask_pth)
#     print(mask.shape)
#     image_new = cv2.resize(image,(w,h),interpolation = cv2.INTER_AREA)
#     mask_new = cv2.resize(mask,(w,h),interpolation = cv2.INTER_NEAREST)
    image_new = image_pad(image,(w,h))
    mask_new = image_pad(mask,(w,h))
    
    image_patch = view_as_windows(image,(pw,ph,3),(pw//2,ph//2,3))
    mask_patch = view_as_windows(mask,(pw,ph,3),(pw//2,ph//2,3))
    
#     print(sid,image_patch.shape,len(mask_patch))
    
    for i,(imgs,msks) in enumerate(zip(image_patch,mask_patch)):

        for j,(img,msk) in enumerate(zip(imgs,msks)):
            
            msk = np.squeeze(msk)
            img = np.squeeze(img)
            
            r,c = np.where(msk[:,:,0] > 0)
            
#             print(len(r))
            
            if len(r)>50:
            
                idx = sid+'_'+str(i)+'_'+str(j)

                img_folder,msk_folder = generate_folder(output_dir,idx)

#                 print(i,j,img_folder,msk_folder)

                img_pth = os.path.join(img_folder,idx+'.jpg')

                cv2.imwrite(img_pth,img)

                mask_generation(msk,msk_folder)
            
        
    
#     folder = os.path.join(input_dir,sid)
#     folder_images = os.path.join(folder,'images')
#     folder_masks = os.path.join(folder,'masks')
    
#     if not os.path.exists(folder):
#         os.mkdir(folder)
#         os.mkdir(folder_images)
#         os.mkdir(folder_masks)
    
#     shutil.copy(img_pth,folder_images)
#     mask_generation(mask_file,folder_masks)


  0%|          | 0/114 [00:00<?, ?it/s]

256 2304
256 2304


  1%|          | 1/114 [00:01<02:16,  1.21s/it]

256 2304
256 2304


  2%|▏         | 2/114 [00:02<02:17,  1.23s/it]

...

 98%|█████████▊| 112/114 [02:32<00:02,  1.49s/it]

256 2304
256 2304


 99%|█████████▉| 113/114 [02:33<00:01,  1.57s/it]

256 2304
256 2304


100%|██████████| 114/114 [02:35<00:00,  1.36s/it]


In [33]:
# generate the data according to kaggle format with pathes for valid and test

import shutil
from skimage.util.shape import view_as_windows
from tqdm import tqdm

# input_dir = r''
input_dir = r'/home/xing/Share/Projects/Sanmed/cell_seg/20201127/valid/DAPI-Low'
output_dir = r'/home/xing/Share/Projects/Sanmed/cell_seg/20201127/Dapi_output/val'
w = 2560
h = 2560
pw = 512
ph = 512

file_list = glob2.glob(os.path.join(input_dir,'*'+'.png'))

for mask_pth in tqdm(file_list):
    
    sid = key_fn(mask_pth)
    
#     print(sid,mask_pth)
    
    # step 1: image readin, resize, and patch generation
    
    img_pth = os.path.join(input_dir,sid+'.jpg')
#     print(img_pth)
    
    image = cv2.imread(img_pth)
    mask = cv2.imread(mask_pth)
#     print(mask.shape)
#     image_new = cv2.resize(image,(w,h),interpolation = cv2.INTER_AREA)
#     mask_new = cv2.resize(mask,(w,h),interpolation = cv2.INTER_NEAREST)
    image_new = image_pad(image,(w,h))
    mask_new = image_pad(mask,(w,h))
    
    image_patch = view_as_windows(image,(pw,ph,3),(pw//2,ph//2,3))
    mask_patch = view_as_windows(mask,(pw,ph,3),(pw//2,ph//2,3))
    
#     print(sid,image_patch.shape,len(mask_patch))
    
    for i,(imgs,msks) in enumerate(zip(image_patch,mask_patch)):

        for j,(img,msk) in enumerate(zip(imgs,msks)):
            
            msk = np.squeeze(msk)
            img = np.squeeze(img)
            
            r,c = np.where(msk[:,:,0] > 0)
            
#             print(len(r))
            
            if len(r)>50:
            
                idx = sid+'_'+str(i)+'_'+str(j)

                img_folder,msk_folder = generate_folder(output_dir,idx)

#                 print(i,j,img_folder,msk_folder)

                img_pth = os.path.join(img_folder,idx+'.jpg')

                cv2.imwrite(img_pth,img)

                mask_generation(msk,msk_folder)

100%|██████████| 10/10 [00:14<00:00,  1.46s/it]


# only pad, resize and save, no crop

In [39]:
# generate the data according to kaggle format with pathes

import shutil
from skimage.util.shape import view_as_windows
from tqdm import tqdm

# input_dir = r''
input_dir = r'/home/xing/Share/Projects/Sanmed/cell_seg/20201127/train/DAPI-Low'
output_dir = r'/home/xing/Share/Projects/Sanmed/cell_seg/20201127/Dapi_output_resize/train'
w = 2560
h = 2560
pw = 512
ph = 512

file_list = glob2.glob(os.path.join(input_dir,'*'+'.png'))

for mask_pth in tqdm(file_list):
    
    sid = key_fn(mask_pth)
    
#     print(sid,mask_pth)
    
    # step 1: image readin, resize, and patch generation
    
    img_pth = os.path.join(input_dir,sid+'.jpg')
#     print(img_pth)
    
    image = cv2.imread(img_pth)
    mask = cv2.imread(mask_pth)
#     print(mask.shape)

    image_new = image_pad(image,(w,h))
    mask_new = image_pad(mask,(w,h))
    
#     image_new = cv2.resize(image_new,(w//2,h//2),interpolation = cv2.INTER_AREA)
#     mask_new = cv2.resize(mask_new,(w//2,h//2),interpolation = cv2.INTER_NEAREST)

    img_folder,msk_folder = generate_folder(output_dir,sid)
    
    img_pth = os.path.join(img_folder,sid+'.jpg')
    
    cv2.imwrite(img_pth,image_new)

    mask_generation(mask_new,msk_folder)
    

100%|██████████| 114/114 [06:41<00:00,  3.52s/it]


In [40]:
# generate the data according to kaggle format with pathes

import shutil
from skimage.util.shape import view_as_windows
from tqdm import tqdm

# input_dir = r''
input_dir = r'/home/xing/Share/Projects/Sanmed/cell_seg/20201127/valid/DAPI-Low'
output_dir = r'/home/xing/Share/Projects/Sanmed/cell_seg/20201127/Dapi_output_resize/val'
w = 2560
h = 2560
pw = 512
ph = 512

file_list = glob2.glob(os.path.join(input_dir,'*'+'.png'))

for mask_pth in tqdm(file_list):
    
    sid = key_fn(mask_pth)
    
#     print(sid,mask_pth)
    
    # step 1: image readin, resize, and patch generation
    
    img_pth = os.path.join(input_dir,sid+'.jpg')
#     print(img_pth)
    
    image = cv2.imread(img_pth)
    mask = cv2.imread(mask_pth)
#     print(mask.shape)

    image_new = image_pad(image,(w,h))
    mask_new = image_pad(mask,(w,h))
    
#     image_new = cv2.resize(image_new,(w//2,h//2),interpolation = cv2.INTER_AREA)
#     mask_new = cv2.resize(mask_new,(w//2,h//2),interpolation = cv2.INTER_NEAREST)

    img_folder,msk_folder = generate_folder(output_dir,sid)
    
    img_pth = os.path.join(img_folder,sid+'.jpg')
    
    cv2.imwrite(img_pth,image_new)

    mask_generation(mask_new,msk_folder)

100%|██████████| 10/10 [00:38<00:00,  3.87s/it]
